In [1]:
import os
from typing import Tuple
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision.io.image import read_image
from torchvision.transforms import v2
import matplotlib.pyplot as plt
from data.utils import walk_path

In [2]:
from utils.helper_functions import StreetHazardsDataModule
from data.shift_dataset import LabelFilter

In [3]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, TQDMProgressBar
from nets.wrapper import Wrapper

In [4]:
# ds_train, ds_val, ds_test = load_shift_segmentation(
#     "./datasets/SHIFT", 512, 6, LabelFilter("4", -1, 0), LabelFilter("4", -1, 0), "ood_pedestrian", 8, .05)
# print(len(ds_train.dataset), len(ds_val.dataset), len(ds_test.dataset))

In [5]:
# ds_train, ds_val, ds_test = load_shift_ood(
#     "./datasets/SHIFT", 512,
#     "./datasets/COCO2014", 256, 4, LabelFilter("4", -1, 0), LabelFilter("4", 10000, 15000), "ood_pedestrian",
#     .7, .9, True, 3, 1, .05)

In [6]:
# ds_train, ds_val, ds_test = load_streethazards_ood(
#     "./datasets/StreetHazards", 512,
#     "./datasets/COCO2014", 256, 4, "normal",
#     .7, .9, True, 3, 1)

In [7]:
dm = StreetHazardsDataModule(
    "./datasets/StreetHazards", 512, 8, "normal", 8)

In [8]:
model = Wrapper("mobilenet", 13, .0001)

In [9]:
model.eval()

Wrapper(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (low_level_features): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): InvertedResidual(
          (conv): Sequential(
            (0): ConvBNReLU(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU6(inplace=True)
            )
            (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (2): InvertedResidual(
          (conv): Sequential(
            (0): ConvBNReLU(
              (0): Conv2d

In [10]:
model.cuda()

Wrapper(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (low_level_features): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): InvertedResidual(
          (conv): Sequential(
            (0): ConvBNReLU(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU6(inplace=True)
            )
            (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (2): InvertedResidual(
          (conv): Sequential(
            (0): ConvBNReLU(
              (0): Conv2d

In [17]:
with torch.no_grad():
    model.model(torch.randn((1, 3, 1260, 800), dtype=torch.float32, device="cuda"))

In [ ]:
ckpt = ModelCheckpoint(
        monitor="val_miou",
        mode="max",
        save_top_k=2,
        filename='{epoch}-{step}-{val_miou:.4f}'
    )

In [ ]:
tr = Trainer(default_root_dir="./test", accelerator="cuda", callbacks=[ckpt, LearningRateMonitor(logging_interval="epoch"),
        TQDMProgressBar(refresh_rate=5)], max_epochs=2)

In [ ]:
tr.fit(model=model, datamodule=dm)